In [1]:
import json
from pathlib import Path
import sys
from functools import partial
from tqdm import tqdm_notebook as tqdm
sys.path.append("../") 

In [2]:
from src.data.utils import read_jsonl, extract_text_by_headers_html, dedup_results, html2text_parser
import torch
from src.models.ir.crossencoder import CrossEncoderIR
from src.models.llm.llama import LlamaLLM
from src.models.baseline_pipeline import BaselinePipeline
from sentence_transformers import CrossEncoder


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [3]:
json_objs = read_jsonl("../data/raw/task1/crag_task_1_v2.jsonl", -1)
json_objs = [json_obj for json_obj in json_objs if json_obj["split"] == 0]
# preprocessing_fn = partial(extract_text_by_headers_html, version="v2", split_sentences=False)
preprocessing_fn = html2text_parser
ir_model = CrossEncoderIR(max_sentences=10, score_threshold=-1)
pipeline = BaselinePipeline(preprocessing_fn, ir_model)

In [4]:
from pathlib import Path

if Path("mnli_dataset.json").is_file():
    with open("mnli_dataset.json") as f:
        dataset = json.load(f)
else:
    dataset = []
    for json_obj in tqdm(json_objs):
        answer, query, search_results, query_time = json_obj["answer"], json_obj["query"], json_obj["search_results"], json_obj["query_time"]
        deduped_search_results = dedup_results(search_results)
        candidates = []
        for i, search_result in enumerate(deduped_search_results):
            # with open(f"htmls/{i}.html", "w") as f:
            #     print(search_result["page_result"], file=f)
            # print(search_result["page_url"])
            for segment in preprocessing_fn(search_result):
                candidates.append(segment)

        dataset.append(
            {
                "query": query,
                "answer": answer,
                "candidates": candidates,
                "query_time": query_time
            }
        )
    with open("mnli_dataset.json", "w") as f:
        json.dump(dataset, f)


/tmp/ipykernel_18814/3656840772.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for json_obj in tqdm(json_objs):


  0%|          | 0/1384 [00:00<?, ?it/s]

In [5]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import AutoModelForSequenceClassification, DebertaV2Tokenizer

model_name = "microsoft/deberta-v2-xlarge-mnli"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

ImportError: 
DebertaV2Converter requires the protobuf library but it was not found in your environment. Checkout the instructions on the
installation page of its repo: https://github.com/protocolbuffers/protobuf/tree/master/python#installation and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.


In [ ]:
s1 = "Kris Jenner has 6 children"
s2 = "Kris Jenner's first marriage was to lawyer Robert Kardashian (who later became widely known for his early legal representation of O. J. Simpson) on July 8, 1978.[74][75] They have four children: daughters Kourtney (born 1979), Kim (born 1980), Khloé (born 1984), and son Rob (born 1987). In April 1991, one month after her divorce from Kardashian, Jenner married her second spouse, retired Olympian Bruce Jenner, who publicly came out as a transgender woman in 2015, taking the name Caitlyn.[77] They have two daughters together: Kendall (born 1995) and Kylie (born 1997)."
input = tokenizer(s1, s2, truncation=True, max_length=512, return_tensors="pt")

In [ ]:
output = output = model(input["input_ids"])  # device = "cuda:0" or "cpu"

In [ ]:
output

SequenceClassifierOutput(loss=None, logits=tensor([[-1.5781,  2.9682, -1.2822]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)